# Fine-tuning Llama 3.1 8B LoRA con Unsloth

> Sígueme en RRSS para apoyar este contenido ❤️: <br>
x -> [@codingmindsetio](https://x.com/codingmindsetio) <br>
YT 🎥 -> [CodingMindset](https://www.youtube.com/@CodingMindsetIO?sub_confirmation=1) <br>
IG 📸 -> [@codingmindset](https://www.instagram.com/codingmindset?igsh=ZGx5aGd4MXBwYmx5&utm_source=qr)

## Instalación e importación de dependencias

Para instalar Unsloth en tu propio PC, sigue las instrucciones de instalación de la página de Github [aquí](https://github.com/unslothai/unsloth?tab=readme-ov-file#-installation-instructions).

In [ ]:
%%capture
# Instala Unsloth, Xformers (Flash Attention) y todos los demás paquetes.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Tenemos que comprobar qué versión de Torch para Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

Comprobamos si tenemos disponible una GPU de Nvidia, si no al importar Unsloth obtendremos un RuntimeError

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Aug 28 07:02:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              44W / 350W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

Importamos todas las dependecias a utilizar

In [ ]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Descarga del modelo

 ### Parámetros

 `max_seq_length`: Al preparar el modelo para su uso, es necesario establecer un límite máximo para la longitud de las secuencias, lo que afecta su capacidad de procesar información contextual. Aunque la versión 3.1 de Llama puede manejar contextos de hasta 128 mil tokens, en este caso optaremos por una configuración más modesta de 2,048 tokens. Esta elección se debe a que utilizar la capacidad máxima requiere considerablemente más recursos computacionales y de VRAM.
 <br>
 <br>
 `dtype`: se refiere al tipo de datos que se utilizará para representar los números en el modelo. Es importante porque afecta la precisión de los cálculos, el uso de memoria y la velocidad de procesamiento.

*   **None**: Detección automática
*   **torch.float32**: Precisión estándar, compatible con la mayoría de GPUs
*   **torch.float16**: Para GPUs Tesla T4, V100
*   **torch.bfloat16**: Para GPUs Ampere y más recientes

`load_in_4bit`: Activa la cuantización de 4 bits para ahorrar memoria. Es opcional (puede ser False)
 <br>
 <br>
`model_name`: En este caso escogemos la versión pre-cuantizada de Llama de 4-bit, debido a que es mucho más ligera (5.4 GB) comprado con la versión original con precisión de 16-bit (16 GB)



In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Preparación LoRA - PEFT (Parameter Efficient Fine Tuning)

LoRA tiene tres parámetros fundamentales:

1. `r` (Rango): Este valor determina el tamaño de las matrices de LoRA. Normalmente, se empieza con un rango de 8, pero puede llegar hasta 256. Un rango mayor permite almacenar más información, pero también aumenta el coste computacional y de memoria. En este caso, hemos optado por un valor de 16.

2. `lora_alpha` (α): Es un factor de escala para las actualizaciones. Alfa influye directamente en cuánto contribuyen los adaptadores (controla la magnitud de la contribución de los adaptadores LoRA a la red principal) y suele establecerse como 1 o 2 veces el valor del rango. En este caso, hemos optado por un valor de 16 (1 * r).

3. `target_modules`: LoRA se puede aplicar a varios componentes del modelo, como los mecanismos de atención (matrices Q, K, V), proyecciones de salida, bloques feed-forward y capas lineales de salida. Aunque inicialmente se centra en los mecanismos de atención, expandir LoRA a otros componentes ha demostrado ser beneficioso. Sin embargo, adaptar más módulos implica un aumento en el número de parámetros entrenables y en las necesidades de memoria. En este caso,  hemos decidido aplicar LoRA a todos los módulos lineales para maximizar la calidad.

Otros parámetros:

`lora_dropout`: Técnica de regularización que desactiva aleatoriamente un porcentaje de conexiones en las matrices de adaptación durante el entrenamiento. Su objetivo principal es prevenir el sobreajuste. Ralentiza ligeramente el entrenamiento por lo que en este caso optamos por no usarlo.

`use_rslora` (Estabilizador de rango): Introduce una modificación en el factor de escala de los adaptadores LoRA. En lugar de usar una proporción de 1/r, emplea 1/√r. Este sutil pero importante cambio tiene dos efectos principales:

1. Estabiliza el proceso de aprendizaje, siendo especialmente beneficioso cuando se utilizan rangos de adaptador más altos.

2. Permite mejorar el rendimiento del fine-tuning a medida que se incrementa el rango del adaptador.

En esencia, rsLoRA busca optimizar el equilibrio entre la capacidad de adaptación y la estabilidad del entrenamiento, especialmente al trabajar con configuraciones de LoRA más complejas.

`use_gradient_checkpointing`: Unsloth se encarga de gestionar el checkpointing del gradiente. Esta técnica consiste en almacenar temporalmente en el disco duro las capas de embedding de entrada y salida, en lugar de mantenerlas constantemente en la memoria de la tarjeta gráfica. El objetivo principal de esta estrategia es optimizar el uso de la VRAM, liberando espacio para otros procesos del modelo durante el entrenamiento.



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Preparación del Dataset y el Tokenizador

Preparamos el tokenizador

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)


Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


En este caso vamos a usar el dataset `"mlabonne/FineTome-100k"`. Usa el formato ShareGPT (dataset), ideal para conversaciones multi-turno. Este formato se procesa para extraer pares de instrucción-respuesta. Luego, los datos se reformatean según una plantilla de chat, como ChatML, que estructura la conversación. ChatML usa tokens especiales para marcar el inicio y fin de cada mensaje.

In [ ]:
def apply_template(examples):
    messages = examples["conversations"]
    text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in messages]
    return {"text": text}

In [ ]:
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
dataset = dataset.map(apply_template, batched=True)

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

## Entrenamiento



Los hiperparámetros clave en el entrenamiento de modelos incluyen:

1. **`packing` (Empaquetado)**: Técnica para combinar múltiples muestras pequeñas en un lote, mejorando la eficiencia del procesamiento.

2. **`learning_rate` (Tasa de aprendizaje)**: Controla la intensidad de las actualizaciones de los parámetros. Debe equilibrarse para evitar un aprendizaje lento o inestable. Usaremos 0.0003 (3e-4)

3. **`lr_scheduler_type` (Planificador de tasa de aprendizaje)**: Ajusta la tasa durante el entrenamiento, generalmente comenzando alta y disminuyendo gradualmente. En este caso nos decantamos por la opcion lineal, una de las opciones más comunes.

4. **`per_device_train_batch_size` (Tamaño del lote)**: Determina cuántas muestras se procesan antes de actualizar los pesos. Lotes más grandes pueden mejorar la estabilidad y velocidad, pero requieren más memoria. El tamaño del lote será 8.

5. **`gradient_accumulation_steps` (Acumulador de gradientes)**: Es una técnica que permite simular un tamaño de lote más grande sin aumentar el uso de memoria. Funciona acumulando gradientes durante varias pasadas hacia adelante y hacia atrás antes de realizar una actualización de los pesos del modelo. Para esta ocasión, con 2 será suficiente.

6. **`num_train_epochs` (Número de épocas)**: Cantidad de veces que el modelo recorre todo el conjunto de datos. Más épocas pueden mejorar el rendimiento, pero también pueden causar sobreajuste. Para ejemplificar usaré 1 step, una vuelta completa a todo el set de entramiento.

7. **`optim` (Optimizador)**: Algoritmo para ajustar los parámetros del modelo. Se recomienda AdamW de 8 bits por su eficiencia en uso de memoria.

8. **`weight_decay` (Decaimiento de pesos)**: Técnica de regularización que penaliza pesos grandes para prevenir el sobreajuste.

9. **`warmup_steps`** (Pasos de calentamiento): Periodo inicial donde la tasa de aprendizaje aumenta gradualmente, ayudando a estabilizar el entrenamiento.


Cada uno de estos hiperparámetros juega un papel crucial en el rendimiento y la eficiencia del entrenamiento, y su ajuste adecuado es fundamental para obtener los mejores resultados.

Otros parámetros:
- `fp16`: Habilita el entrenamiento en precisión mixta de 16 bits (si bfloat16 no es compatible).

- `bf16`: Habilita el entrenamiento en bfloat16 (si es compatible).

- `seed`: Semilla para la generación de números aleatorios (puede ser cualquier número).

In [ ]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

Ejecutamos el entranamiento

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 29,003 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 1,813
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.197700
2,1.138300
3,1.149200
4,1.133300
5,1.001500
6,1.020000
7,0.939100
8,0.949000
9,0.893400
10,0.944600


Step,Training Loss
1,1.197700
2,1.138300
3,1.149200
4,1.133300
5,1.001500
6,1.020000
7,0.939100
8,0.949000
9,0.893400
10,0.944600


## Inferencia

`FastLanguageModel.for_inference()` nos proporciona de manera nativa una inferencia el doble de rápida

In [ ]:
model = FastLanguageModel.for_inference(model)

Finalmente. realizamos la inferencia:

In [ ]:
messages = [
    {"from": "human", "value": "Which number is larger: 9.9 or 9.11"},
]

In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

In [ ]:
text_streamer = TextStreamer(tokenizer)

In [ ]:
outputs = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

<|im_start|>user
Which number is larger: 9.9 or 9.11<|im_end|>
<|im_start|>assistant
To compare two decimal numbers, we need to compare their decimal places. The decimal place is the place where the decimal point is located. In this case, both numbers have one decimal place.

To compare the numbers, we need to compare the digits to the right of the decimal point. In this case, the digits to the right of the decimal point are 9 and 1.

Since 9 is greater than 1, 9.9 is larger than 9.11.

Therefore, the answer is 9.9.<|im_end|>


## Guardado del modelo

Ahora es el momento de guardar nuestro modelo entrenado. Es importante recordar que, debido a la naturaleza de LoRA y QLoRA, lo que realmente hemos entrenado no es el modelo completo, sino un conjunto de adaptadores.

Unsloth nos ofrece tres métodos para guardar nuestro trabajo:

1. `lora`: Este método guarda únicamente los adaptadores, sin el modelo base.

2. `merged_16bit`: Combina los adaptadores con el modelo base y los guarda en precisión de 16 bits.

3. `merged_4bit`: Similar al anterior, pero guarda el modelo combinado en precisión de 4 bits, lo que resulta en un archivo más compacto.

La elección del método dependerá de nuestras necesidades específicas de almacenamiento y uso futuro del modelo. En nuestro caso escogemos la opción de 16bit para mayor precisión.

In [ ]:
# Guarda el modelo en local
model.save_pretrained_merged("lora_model", tokenizer, save_method="merged_16bit")


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 52.65 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 56.56it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Guardamos el modelo en nuestro repositorio de HuggingFace

In [ ]:
model.push_to_hub_merged("3lv27/CodingMindsetTunnedLlama-3.1-8B", tokenizer, save_method="merged_16bit", token="")

Unsloth: You are pushing to hub, but you passed your HF username = 3lv27.
We shall truncate 3lv27/CodingMindsetTunnedLlama-3.1-8B to CodingMindsetTunnedLlama-3.1-8B
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 52.65 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 55.83it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/3lv27/CodingMindsetTunnedLlama-3.1-8B


Unsloth ofrece una funcionalidad adicional muy útil: la conversión directa de tu modelo al formato GGUF. Este formato de cuantización, diseñado originalmente para llama.cpp, es ampliamente compatible con diversos motores de inferencia, como LM Studio, Ollama, etc.

Una de las ventajas del formato GGUF es que permite especificar diferentes niveles de precisión. Aprovecharemos esta característica para generar múltiples versiones cuantizadas del modelo. Concretamente, crearemos versiones en las siguientes precisiones: `q2_k`, `q3_k_m`, `q4_k_m`, `q5_k_m`, `q6_k` y `q8_0`.

Todas estas versiones cuantizadas se subirán a Hugging Face. El repositorio contendrá todos nuestros archivos GGUF generados.

Esta approach nos permite ofrecer una gama de opciones que equilibran el tamaño del modelo y su precisión, adaptándose así a diferentes necesidades de implementación.

In [ ]:
model.push_to_hub_gguf(
        "3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF", # Cambia hf por tu nombre de usuario / nombre del repo
        tokenizer,
        quantization_method = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"],
        token = "", # Obtén tu token de hf en https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 58.45 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 27.46it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q2_k', 'q3_k_m', 'q4_k_m', 'q5_k_m', 'q6_k', 'q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at 3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF into bf16 GGUF format.
The output location will be ./3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: CodingMindsetTunnedLlama-3.1-8B-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
I

...

Saved GGUF to https://huggingface.co/3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q6_K.gguf:   0%|          | 0.00/6.60G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF
Saved Ollama Modelfile to https://huggingface.co/3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF


## Nota importante

Si has llegado hasta aquí, espero que este tutorial detallado te haya sido de gran ayuda. Si ha sido así, te agradecería mucho que me ayudases apoyando el contenido para poder seguir creando estos tutoriales tan detallados, ya que llevan muchas horas de trabajo.

Apoya el contenido dejándo tu like y un comentario 🙏🙏❤️

Y no olvides seguirme en mis redes!

x -> [@codingmindsetio](https://x.com/codingmindsetio) <br>
YT 🎥 -> [CodingMindset](https://www.youtube.com/@CodingMindsetIO?sub_confirmation=1) <br>
IG 📸 -> [@codingmindset](https://www.instagram.com/codingmindset?igsh=ZGx5aGd4MXBwYmx5&utm_source=qr)

Muchas gracias de corazón! ❤️

In [ ]:
from functools import partial

quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]

def push_to_hub_gguf_with_quant(model, tokenizer, hub_path, quant):
    model.push_to_hub_gguf(hub_path, tokenizer, quant, token="")

push_model = partial(push_to_hub_gguf_with_quant, model, tokenizer, "3lv27/CodingMindsetTunnedLlama-3.1-8B-GGUF")

list(map(push_model, quant_methods))